##

In [2]:
from data_cleaning import DatabaseConnector
from data_extraction import DataExtractor
import pandas as pd
new_db_creds = './db_creds_local.yaml' # write the file path to the file containing database credentials. Refer to README.md
db_conn = DatabaseConnector(new_db_creds)
data_extrc = DataExtractor()

## Task 1: How many stores does the business have and in which countries?
The Operations team would like to know which countries we currently operate in and which country now has the most stores. 

In [3]:
query = '''
            SELECT 
                country_code, 
                COUNT(country_code) AS store_count
            FROM dim_store_details
            GROUP BY country_code
            ORDER BY store_count DESC;
            '''
print(data_extrc.read_rds_table(query, db_conn))

  country_code  store_count
0           GB          266
1           DE          141
2           US           34


## 2. Which locations currently have the most stores?
The business stakeholders would like to know which locations currently have the most stores. They would like to close some stores before opening more in other locations. Find out which locations have the most stores currently.

In [4]:
query = '''
            SELECT 
                locality, 
                COUNT(locality) AS store_count
            FROM dim_store_details
            GROUP BY locality
            ORDER BY store_count DESC
            LIMIT 7;
            '''
print(data_extrc.read_rds_table(query, db_conn))


       locality  store_count
0    Chapletown           14
1        Belper           13
2        Bushey           12
3        Exeter           11
4      Arbroath           10
5  High Wycombe           10
6    Rutherglen           10


## 3. Which months produced the largest amount of sales?
Query the database to find out which months have produced the most sales.

In [5]:
query = '''
            SELECT ROUND(
                            CAST(SUM(dim_products.product_price * orders_table.product_quantity) AS numeric), 2
                        ) AS total_sales,
                dim_date_times.month
            FROM orders_table
            INNER JOIN dim_date_times ON dim_date_times.date_uuid = orders_table.date_uuid
            INNER JOIN dim_products ON dim_products.product_code = orders_table.product_code
            GROUP BY dim_date_times.month
            ORDER BY total_sales DESC
            LIMIT 6;
        '''
print(data_extrc.read_rds_table(query, db_conn))

  total_sales month
0   673295.68     8
1   668041.45     1
2   657335.84    10
3   650321.43     5
4   645741.70     7
5   645463.00     3


## 4. How many sales are coming from online?
The company is looking to increase its online sales. They want to know how many sales are happening online vs offline. Calculate how many products were sold and the amount of sales made for online and offline purchases.

In [6]:
query = '''
            SELECT
                    COUNT(date_uuid) AS number_of_sales,
                    SUM(orders.product_quantity),
                    CASE
                        WHEN store_type IN ('Web Portal')
                            THEN 'Web'
                        WHEN store_type IN ('Super Store', 'Local', 'Outlet', 'Mall Kiosk')
                            THEN 'Offline'
                    END AS location
            FROM orders_table AS orders
            INNER JOIN dim_store_details AS stores ON stores.store_code = orders.store_code
            GROUP BY location
            ORDER BY number_of_sales ASC
        '''
print(data_extrc.read_rds_table(query, db_conn))

   number_of_sales     sum location
0            26957  107739      Web
1            93166  374047  Offline


## 5. What percentage of sales come through each type of store? (PERCENTAGES IS WRONG)
The sales team wants to know which of the different store types is generated the most revenue so they know where to focus. Find out the total and percentage of sales coming from each of the different store types.

In [7]:
query = '''
            SELECT
                store_type,
                ROUND(
                    CAST(SUM(products.product_price * orders.product_quantity) AS numeric), 2
                ) AS total_sales,
                ROUND(
                    CAST(100 * (SUM(products.product_price * orders.product_quantity) / 
                                (SELECT SUM(products.product_price * orders.product_quantity)
                                FROM orders_table AS orders
                                INNER JOIN dim_products AS products ON products.product_code = orders.product_code
                                )
                            ) AS numeric), 2
                ) AS "percentage_total(%)"
            FROM
                orders_table AS orders
                INNER JOIN dim_date_times AS date_times ON date_times.date_uuid = orders.date_uuid
                INNER JOIN dim_products AS products ON products.product_code = orders.product_code
                INNER JOIN dim_store_details AS store_details ON store_details.store_code = orders.store_code
            GROUP BY
                store_details.store_type
            ORDER BY
                total_sales DESC;
        '''
print(data_extrc.read_rds_table(query, db_conn))

    store_type total_sales percentage_total(%)
0        Local  3440896.52               44.56
1   Web Portal  1726547.05               22.36
2  Super Store  1224293.65               15.85
3   Mall Kiosk   698791.61                9.05
4       Outlet   631804.81                8.18


## 6. Which month in each year produced the highest cost of sales?
The company stakeholders want assurances that the company has been doing well recently. Find which months in which years have had the most sales historically.


In [8]:
query = '''
            SELECT
                    ROUND(
                        CAST(SUM(products.product_price * orders.product_quantity) AS numeric), 2
                    ) AS total_sales,
                    dates.year,
                    dates.month
            FROM orders_table AS orders
            INNER JOIN dim_products AS products ON products.product_code = orders.product_code
            INNER JOIN dim_date_times AS dates ON dates.date_uuid = orders.date_uuid
            GROUP BY dates.year, dates.month
            ORDER BY total_sales DESC
            LIMIT 10;
        '''
print(data_extrc.read_rds_table(query, db_conn))

  total_sales  year month
0    27936.77  1994     3
1    27356.14  2019     1
2    27091.67  2009     8
3    26679.98  1997    11
4    26310.97  2018    12
5    26277.72  2019     8
6    26236.67  2017     9
7    25798.12  2010     5
8    25648.29  1996     8
9    25614.54  2000     1


## 7. What is our staff headcount?
The operations team would like to know the overall staff numbers in each location around the world. Perform a query to determine the staff numbers in each of the countries the company sells in.

In [9]:
query = '''
            SELECT
                    SUM(staff_numbers) AS total_staff_numbers,
                    country_code
            FROM dim_store_details
            GROUP BY country_code
            ORDER BY total_staff_numbers DESC
        '''
print(data_extrc.read_rds_table(query, db_conn))

   total_staff_numbers country_code
0                13307           GB
1                 6087           DE
2                 1384           US


## 8. Which German store type is selling the most?
The sales team is looking to expand their territory in Germany. Determine which type of store is generating the most sales in Germany.

In [10]:
query = '''
            SELECT
                ROUND(
                    CAST(SUM(products.product_price * orders.product_quantity) AS numeric), 2
                ) AS total_sales,
                store_type,
                stores.country_code
            FROM orders_table AS orders
            INNER JOIN dim_products AS products ON products.product_code = orders.product_code
            INNER JOIN dim_store_details AS stores ON stores.store_code = orders.store_code
            GROUP BY stores.store_type, stores.country_code
            HAVING country_code = 'DE'
            ORDER BY total_sales ASC;
        '''
print(data_extrc.read_rds_table(query, db_conn))

  total_sales   store_type country_code
0   198373.57       Outlet           DE
1   247634.20   Mall Kiosk           DE
2   384625.03  Super Store           DE
3  1109909.59        Local           DE


## 9. How quickly is the company making sales?
Sales would like the get an accurate metric for how quickly the company is making sales. Determine the average time taken between each sale grouped by year.

In [11]:
query = '''
        WITH timestamp_converted AS (
                SELECT
                        year,
                        TO_TIMESTAMP(
                                CONCAT(year, '-', month, '-', day, ' ', timestamp),
                                'YYYY-MM-DD HH24:MI:SS'
                        ):: TIMESTAMP WITHOUT TIME ZONE AS timestamp

                FROM dim_date_times
                ORDER BY timestamp ASC
        ), get_end_time AS (
                SELECT
                        year,
                        timestamp,
                        LEAD (timestamp) OVER (
                                PARTITION BY year
                                ORDER BY timestamp ASC
                        ) AS end_time
                FROM timestamp_converted
                ORDER BY timestamp
        ), calc_avg_time AS (
                SELECT
                        year,
                        AVG(end_time - timestamp) AS actual_time_taken
                FROM get_end_time
                GROUP BY year
                ORDER BY actual_time_taken DESC
        )

        SELECT
                year,
                CONCAT (
                        'hours: ', EXTRACT(HOUR FROM actual_time_taken), ', ',
                        'minutes: ',EXTRACT(MINUTE FROM actual_time_taken), ', ',
                        'seconds: ',TRUNC(EXTRACT(SECOND FROM actual_time_taken)), ', ',
                        'milliseconds: ',EXTRACT(MILLISECONDS FROM actual_time_taken)
                ) AS actual_time_taken
        FROM calc_avg_time
        LIMIT 5;
        '''
print(data_extrc.read_rds_table(query, db_conn))

   year                                  actual_time_taken
0  2013  hours: 2, minutes: 17, seconds: 13, millisecon...
1  1993  hours: 2, minutes: 15, seconds: 35, millisecon...
2  2002  hours: 2, minutes: 13, seconds: 39, millisecon...
3  2008  hours: 2, minutes: 13, seconds: 3, millisecond...
4  2022  hours: 2, minutes: 13, seconds: 2, millisecond...
